In [1]:
import tensorflow as tf
from tensorflow.contrib.data import Dataset
import glob
import sys
import scipy
import numpy as np

In [2]:
# helper functions
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
#functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
data_path = "cifar10-hw2/"
X_train, y_train = get_train_data(data_path)

y_train = y_train.reshape(50000,)
X_train = np.swapaxes(X_train,1,0)
print(X_train.shape)
X_train = np.asarray(X_train,dtype = np.float32)
y_train = np.asarray(y_train,dtype = np.int32)

{'airplane': 0, 'frog': 6, 'ship': 8, 'bird': 2, 'dog': 5, 'automobile': 1, 'truck': 9, 'cat': 3, 'horse': 7, 'deer': 4}


/home/leo/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
(50000, 3072)


In [23]:
#split into train and test segments
X_test = X_train[-10000:,:]
X_train = X_train[:40000]
y_test = y_train[-10000:]
y_train = y_train[:40000]

print(X_test.shape)
print(X_train.shape)
print(y_test.shape)



(10000, 3072)
(40000, 3072)
(10000,)


In [59]:
LOGDIR = "tmp/cifar-w_validation_data/"
tf.reset_default_graph()
def train_input_fn(features, labels, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    # step 3: parse every image in the dataset using `map`
    dataset = dataset.map(_parse_function)
    print(dataset)
    #dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    print((features,labels))

    return features, labels
"""
def _parse_function(filename,label):
    print(filename)
    print("HELLO")
    image_string = tf.read_file(filename)
    print(image_string)
    image_decoded = tf.image.decode_png(image_string, channels=3)
    image_decoded = tf.reshape(image_decoded, [3072])

    image = tf.cast(image_decoded, tf.float32)
    print(image)
    label = tf.reshape(label, [1])
    return image,label"""

def my_model_fn(features, labels, mode, params): 

    input_layer = tf.reshape(features["x"],[-1,32,32,3])
    tf.summary.image("image",input_layer,3)
    with tf.name_scope("conv1"):
    #conv layer 1
        conv1 = tf.layers.conv2d(
          inputs=input_layer,
          filters=32,
          kernel_size=[5,5],
          padding="same",
          activation=tf.nn.relu,
          name = "conv1")
        tf.summary.histogram("conv1",conv1)
        
    #with tf.variable_scope("conv1") as scope:
        #tf.get_variable_scope().reuse_variables()

       # kernel = tf.get_variable('kernel')
        
        """W1_a = kernel
        W1pad= tf.zeros([5, 5, 3, 4]) # [5, 5, 3, 4]  - four zero kernels for padding
        print(W1pad)
        W1_b = tf.concat([W1pad,W1_a],3)   # [5, 5, 3, 36]
        print(W1_b)
        W1_c = tf.split(W1_b, 36, 3)         # 36 x [5, 5, 3, 1]
        W1_row0 = tf.concat(W1_c[0:6], 0)    # [30, 5, 3, 1]
        W1_row1 = tf.concat(W1_c[6:12],0)   # [30, 5, 3, 1]
        W1_row2 = tf.concat(W1_c[12:18],0)  # [30, 5, 3, 1]
        W1_row3 = tf.concat(W1_c[18:24],0)  # [30, 5, 3, 1]
        W1_row4 = tf.concat(W1_c[24:30],0)  # [30, 5, 3, 1]
        W1_row5 = tf.concat(W1_c[30:36],0)  # [30, 5, 3, 1]
        W1_d = tf.concat([W1_row0, W1_row1, W1_row2, W1_row3, W1_row4, W1_row5],1) # [30, 30, 3, 1]
        W1_e = tf.reshape(W1_d, [1, 30, 30, 3])
        Wtag = tf.placeholder(tf.string, None)
        #tf.image_summary(Wtag, W1_e)
        tf.summary.image(Wtag,W1_e)"""

        
    with tf.name_scope("pool1"):
        #pooling layer 1
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size = [2,2],strides = 2)
        tf.summary.histogram("pool1",pool1)
        
    #Convolutional Layer #2 and Pooling Layer #2
    with tf.name_scope("conv2"):
        conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [5,5],
                                 padding = 'same', activation = tf.nn.relu)
        tf.summary.histogram("conv2",conv2)
        
    with tf.name_scope("pool2"):
        pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2],strides = 2)
        tf.summary.histogram("pool2",pool2)
        
    with tf.name_scope("pool2_flat"):
        #dense layer
        pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
        tf.summary.histogram("pool2_flat",pool2_flat)
        
    with tf.name_scope("dense"):
        dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
        tf.summary.histogram("dense",dense)
    
    with tf.name_scope("dropout"):
        dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
        tf.summary.histogram("dropout",dropout)
        
    with tf.name_scope("logits"):
        logits = tf.layers.dense(inputs = dropout, units = 10)
        tf.summary.histogram("logits",logits)
    
    
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits,name = "softmax_tensor")
        }
    
    with tf.name_scope("train-accuracy"):
        train_prediction = tf.nn.softmax(logits)
        #train_prediction = tf.cast(train_prediction, tf.int64)
        cast_labels = tf.cast(labels,tf.int64)
        correct_prediction = tf.equal(tf.argmax(train_prediction, 1), cast_labels)
      
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("train-accuracy", accuracy)

    
    if(mode == tf.estimator.ModeKeys.PREDICT):
        predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits,name = "softmax_tensor")
        }
        return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)


    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
    
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

        train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    

    
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR)
    writer.add_graph(sess.graph)
    
                     

In [64]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[3072], dtype=tf.float32)]
tf.reset_default_graph()
#set up classifier
classifier = tf.estimator.Estimator(
    model_fn = my_model_fn,
    model_dir = LOGDIR,
    params={
        'feature_columns' : feature_columns,
        #choose between 10 classes
        'n_classes':10,
    }
)
#tensors_to_log = {"probabilities": "softmax_tensor"}
#logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)



train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":X_train},
    y = y_train,
    batch_size = 100,
    num_epochs = None,
    shuffle = True
    )
classifier.train(input_fn=train_input_fn, steps = None)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_task_id': 0, '_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_is_chief': True, '_session_config': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_service': None, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f607ba17ac8>, '_task_type': 'worker', '_model_dir': 'tmp/cifar-w_validation_data/', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from tmp/cifar-w_validation_data/model.ckpt-150343
INFO:tensorflow:Saving checkpoints for 150344 into tmp/cifar-w_validation_data/model.ckpt.
INFO:tensorflow:step = 150344, loss = 0.5265207
INFO:tensorflow:global_step/sec: 38.0635
INFO:tensorflow:step = 150444, loss = 0.43639228 (2.628 sec)
INFO:tensorflow:global_step/sec: 38.

INFO:tensorflow:step = 157644, loss = 0.4332565 (2.939 sec)
INFO:tensorflow:global_step/sec: 29.5933
INFO:tensorflow:step = 157744, loss = 0.42717406 (3.383 sec)
INFO:tensorflow:global_step/sec: 29.8726
INFO:tensorflow:step = 157844, loss = 0.5066179 (3.343 sec)
INFO:tensorflow:global_step/sec: 28.0102
INFO:tensorflow:step = 157944, loss = 0.4317413 (3.570 sec)
INFO:tensorflow:global_step/sec: 26.4112
INFO:tensorflow:step = 158044, loss = 0.39960045 (3.787 sec)
INFO:tensorflow:global_step/sec: 27.2716
INFO:tensorflow:step = 158144, loss = 0.38132858 (3.667 sec)
INFO:tensorflow:global_step/sec: 30.2685
INFO:tensorflow:step = 158244, loss = 0.5367272 (3.303 sec)
INFO:tensorflow:global_step/sec: 33.9105
INFO:tensorflow:step = 158344, loss = 0.32594842 (2.949 sec)
INFO:tensorflow:global_step/sec: 29.1552
INFO:tensorflow:step = 158444, loss = 0.39563864 (3.436 sec)
INFO:tensorflow:global_step/sec: 27.1101
INFO:tensorflow:step = 158544, loss = 0.30216426 (3.683 sec)


KeyboardInterrupt: 

In [65]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": X_test},
  y=y_test,
  num_epochs=1,
  shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-02-25-02:17:17
INFO:tensorflow:Restoring parameters from tmp/cifar-w_validation_data/model.ckpt-150344
INFO:tensorflow:Finished evaluation at 2018-02-25-02:17:19
INFO:tensorflow:Saving dict for global step 150344: accuracy = 0.0975, global_step = 150344, loss = 6.9776187
{'loss': 6.9776187, 'accuracy': 0.0975, 'global_step': 150344}


In [70]:
X_test = get_images('cifar10-hw2/test')
X_test = X_test.T
X_test = np.asarray(X_test,np.float32)
print(X_test.shape)
y_test = ['airplane', 'frog', 'ship', 'bird', 'dog',
         'automobile', 'truck', 'cat', 'horse', 'deer']
y_test = np.asarray(y_test)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x" : X_test},
    y = None,
    shuffle = False
    
    )
preds = classifier.predict(input_fn = predict_input_fn)
print(preds)

/home/leo/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


Loaded 10000/10000
(10000, 3072)
<generator object Estimator.predict at 0x7f607b2eb830>
